In [1]:
import os
import tempfile
from pathlib import Path

import panel as pn
import param
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

TTL = 1800  # 30 minutes

WIDGET_MAX_WIDTH = 600


class VariableNotFound(Exception):
    def __init__(self, key: str) -> None:
        super().__init__(f"The __key '{key}' is not a supported variable!")


class EnvironmentWidgetBase(pn.viewable.Viewer):

    message_alert: str = param.String(
        (
            "**Protect your secrets!** Make sure you trust "
            "the publisher of this app before entering your secrets."
        ),
        doc="""An Alert message to display to the user to make them handle their secrets
        securely. If not set, then no Alert is displayed""",
    )

    variables_not_set = param.List(
        constant=True, doc="A list of the variables with no value"
    )
    variables_set = param.List(
        constant=True, doc="A list of the variables with a value"
    )

    def __init__(self, **params):
        self._variables = self._get_variables()

        for variable in self._variables:
#             You can write the actual API key to call the LLM at the "placeholder"
            params[variable] = params.get(variable, os.environ.get(variable, "placeholder"))

        layout_params = {}
        for variable, value in params.items():
            if variable in pn.Column.param:
                layout_params[variable] = value
        for variable in layout_params:
            params.pop(variable)

        super().__init__(**params)

        self._layout = self._create_layout(**layout_params)

    def __panel__(self):
        return self._layout

    def _get_variables(self):
        return tuple(
            key for key in self.param if key not in EnvironmentWidgetBase.param
        )

    def _create_layout(self, **params):
        self._update_missing_variables(None)
        if not self.variables_not_set:
            return pn.Column(height=0, width=0, margin=0, sizing_mode="fixed")

        layout = pn.Column(**params)
        if self.message_alert:
            alert = pn.pane.Alert(
                self.message_alert,
                alert_type="danger",
                sizing_mode="stretch_width",
            )
            layout.append(alert)

        for key in self.variables_not_set:
            parameter = self.param[key]
            input_widget = pn.widgets.PasswordInput.from_param(
                parameter,
                max_width=WIDGET_MAX_WIDTH,
                sizing_mode="stretch_width",
                align="center",
            )

            pn.bind(self._update_missing_variables, input_widget, watch=True)
            layout.append(input_widget)
        return layout

    def _update_missing_variables(self, _):
        missing = []
        not_missing = []
        for key in self._variables:
            if not getattr(self, key):
                missing.append(key)
            else:
                not_missing.append(key)
        with param.edit_constant(self):
            self.variables_not_set = sorted(missing)
            self.variables_set = sorted(not_missing)

    def get(self, __key: str, default: str) -> str:
        if __key not in self._variables:
            raise VariableNotFound(key=__key)
        return getattr(self, __key) or default

    def __getitem__(self, key):
        value = self.get(key, "")
        if not value:
            raise VariableNotFound(key=key)
        return value

pn.extension()


@pn.cache(ttl=TTL)
def _get_texts(pdf):
    # loading documents
    with tempfile.NamedTemporaryFile("wb", delete=False) as f:
        f.write(pdf)
    file_name = f.name
    loader = PyPDFLoader(file_name)
    documents = loader.load()

    # splitting the documents into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    return text_splitter.split_documents(documents)

@pn.cache(ttl=TTL)
def _get_vector_db(pdf, openai_api_key):
    texts = _get_texts(pdf)
    # selecting which embeddings we want to use
    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
    # creating the vectorestore to use as the index
    return Chroma.from_documents(texts, embeddings)


@pn.cache(ttl=TTL)
def _get_retriever(pdf, openai_api_key: str, number_of_chunks: int):
    db = _get_vector_db(pdf, openai_api_key)
    return db.as_retriever(
        search_type="similarity", search_kwargs={"k": number_of_chunks}
    )


@pn.cache(ttl=TTL)
def _get_retrieval_qa(
    pdf: bytes, number_of_chunks: int, chain_type: str, openai_api_key: str
):
    retriever = _get_retriever(pdf, openai_api_key, number_of_chunks)
    return RetrievalQA.from_chain_type(
        llm=OpenAI(openai_api_key=openai_api_key),
        chain_type=chain_type,
        retriever=retriever,
        return_source_documents=True,
        verbose=True,
    )


def _get_response(contents):
    qa = _get_retrieval_qa(
        state.pdf, state.number_of_chunks, state.chain_type, environ.OPENAI_API_KEY
    )
    response = qa({"query": contents})
    chunks = []

    for chunk in response["source_documents"][::-1]:
        name = f"Chunk {chunk.metadata['page']}"
        content = chunk.page_content
        chunks.insert(0, (name, content))
    return response, chunks


# Defining the Application State
class EnvironmentWidget(EnvironmentWidgetBase):
    OPENAI_API_KEY: str = param.String()


class State(param.Parameterized):
    pdf: bytes = param.Bytes()
    number_of_chunks: int = param.Integer(default=2, bounds=(1, 5), step=1)
    chain_type: str = param.Selector(
        objects=["stuff", "map_reduce", "refine", "map_rerank"]
    )


environ = EnvironmentWidget()
state = State()

# Defining the widgets
pdf_input = pn.widgets.FileInput.from_param(state.param.pdf, accept=".pdf", height=50)
text_input = pn.widgets.TextInput(placeholder="First, upload a PDF!")
chain_type_input = pn.widgets.RadioButtonGroup.from_param(
    state.param.chain_type,
    orientation="vertical",
    sizing_mode="stretch_width",
    button_type="primary",
    button_style="outline",
)

# Defining and configuring of the ChatInterface

def _get_validation_message():
    pdf = state.pdf
    openai_api_key = environ.OPENAI_API_KEY
    if not pdf:
        return "Please first upload a PDF!"
    return ""


def _send_not_ready_message(chat_interface) -> bool:
    message = _get_validation_message()

    if message:
        chat_interface.send({"user": "System", "object": message}, respond=False)
    return bool(message)


async def respond(contents, user, chat_interface):
    if _send_not_ready_message(chat_interface):
        return
    if chat_interface.active == 0:
        chat_interface.active = 1
        chat_interface.active_widget.placeholder = "Ask questions here!"
        yield {"user": "OpenAI", "object": "Let's chat about the PDF!"}
        return

    response, documents = _get_response(contents)
    pages_layout = pn.Accordion(*documents, sizing_mode="stretch_width", max_width=800)
    answers = pn.Column(response["result"], pages_layout)

    yield {"user": "OpenAI", "object": answers}


chat_interface = pn.chat.ChatInterface(
    callback=respond,
    sizing_mode="stretch_width",
    widgets=[pdf_input, text_input],
    disabled=True,
)

@pn.depends(state.param.pdf, environ.param.OPENAI_API_KEY, watch=True)
def _enable_chat_interface(pdf, openai_api_key):
    if pdf and openai_api_key:
        chat_interface.disabled = False
    else:
        chat_interface.disabled = True


_send_not_ready_message(chat_interface)


template = pn.template.BootstrapTemplate(
    sidebar=[
        environ,
        state.param.number_of_chunks,
        "Chain Type:",
        chain_type_input,
    ],
    main=[chat_interface],
)
template.servable()

BootstrapTemplate
    [js_area] HTML(None, height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] BootstrapTemplateActions()
    [browser_info] BrowserInfo()
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [main-1647265149264] ChatInterface(_button_data={'send': _ChatButtonData(i...}, _buttons={'send': Button(disabled=T...}, _input_container=Row, _input_layout=Tabs, _placeholder=ChatMessage, _widgets={'Pdf': FileInput(accept='...}, callback=<function respond a..., disabled=True, show_button_name=True, sizing_mode='stretch_width', widgets=[FileInput(accept='.pdf', ...])
        [0] ChatMessage(str, avatar='⚙️', reaction_icons=ChatReactionIcons, timestamp=datetime.datetime(2024, ..., user='System')
    [nav-1647265143184] Column(height=0, sizing_mode='fixed', width=0)
    [nav-1647264494352] IntSlider(end=5, name='Number of chunks', start=1, value=2)
    [nav-1647264496016] Markdown(str)
    [nav-1647265031568] RadioButtonGroup(button_style='outline', button_type='primary', name='Chain type', options=OrderedDict([('stuff', ...]), orientation='vertical', sizing_mode='stretch_width', value='stuff')

#### Since I used OpenAI's GPT 3.5-Turbo as LLM which isn't available for free, therefore to run this code enter an API key at the "placeholder" in the code.

### After running the above cell.
#### Kindly open anaconda/python command prompt and give the command "panel serve ChatbotNLP.ipynb".
#### Copy the link given in the response and search it in a browser.
#### This will open a web application of the implemented chatbot.